In [2]:
from final_utils import *

In [3]:
root_path = generate_root_path()

In [ ]:
#importing denoiser
import import_ipynb
%cd Denoiser
from denoiser import Denoiser
%cd ..

## Creating model

In [9]:
#creating feature extractors
model_gap = model_inceptionresnet_multigap()
model_CNN = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b7/feature-vector/1",trainable=False) ])

#creating dense and loading the weights
model = fc_model_softmax(input_num=9744)
weights_path = f'models/Softmax/MG_CNN/model_fc_softmax_MG_8k_B7_1k_600x600.hdf5'
model.load_weights(weights_path)

#loading pca models
pca_mg = pk.load(open('models/PCA/PCA_MG_8464_auto.pkl','rb'))
pca_cnn = pk.load(open('models/PCA/PCA_CNN_1280_auto.pkl','rb'))

## Giving path of folder

In [6]:
good_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'good', '*'))
bad_imgs_path = glob(os.path.join(root_path, 'Data', 'benchmark', 'images', 'bad', '*'))

good_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'benchmark2', 'images', 'good', '*'))
bad_imgs_path_2 = glob(os.path.join(root_path, 'Data', 'benchmark2', 'images', 'bad', '*'))

good_alm_imgs_path = glob(os.path.join(root_path, 'Data', 'splitted', 'bench', 'images', 'good', '*'))
bad_alm_imgs_path = glob(os.path.join(root_path, 'Data', 'splitted', 'bench', 'images', 'bad', '*'))

good_rand_imgs_path = glob(os.path.join(root_path, 'Data', 'custom_data', 'images', 'good', '*'))
bad_rand_imgs_path = glob(os.path.join(root_path, 'Data', 'custom_data', 'images', 'bad', '*'))

paths_bench = good_imgs_path + bad_imgs_path
labels_bench = np.concatenate((np.ones(len(good_imgs_path)) ,np.zeros(len(bad_imgs_path))))
labels_bench_joint = np.concatenate((np.ones(6*len(good_imgs_path)) ,np.zeros(6*len(bad_imgs_path))))

paths_bench_2 = good_imgs_path_2 + bad_imgs_path_2
labels_bench_2 = np.concatenate((np.ones(len(good_imgs_path_2)) ,np.zeros(len(bad_imgs_path_2))))
labels_bench_joint_2 = np.concatenate((np.ones(6*len(good_imgs_path_2)) ,np.zeros(6*len(bad_imgs_path_2))))

paths_alm_bench = good_alm_imgs_path + bad_alm_imgs_path
labels_alm_bench = np.concatenate((np.ones(len(good_alm_imgs_path)) ,np.zeros(len(bad_alm_imgs_path))))
labels_alm_bench_joint = np.concatenate((np.ones(6*len(good_alm_imgs_path)) ,np.zeros(6*len(bad_alm_imgs_path))))

paths_rand_bench = good_rand_imgs_path + bad_rand_imgs_path
labels_rand_bench = np.concatenate((np.ones(len(good_rand_imgs_path)) ,np.zeros(len(bad_rand_imgs_path))))
labels_rand_bench_joint = np.concatenate((np.ones(6*len(good_rand_imgs_path)) ,np.zeros(6*len(bad_rand_imgs_path))))

#### Necessary functions

In [7]:
def calc_acc(labels, predicted):
    acc = np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)
    return np.round(acc * 100, 2)

def add_gauss_noise(img, noise_param):
    gauss = np.random.normal(0, noise_param, img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    # Add the Gaussian noise to the image
    img = cv2.add(img, gauss)
    return img

def sharpness(img):
    kernel = np.array([[-1,-1,-1], 
                       [-1, 9,-1],
                       [-1,-1,-1]])
    img = cv2.filter2D(img, -1, kernel)
    return img

def denoiser(img):
    img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)     
    return img

def read_img(path, preprocess=True, resize_func=None, size=None, for_all=False, aug_method=None):
#     im = Image.open(path).convert('RGB')
#     x = img_to_array(im)
#     im.close()
    x = cv2.imread(path)
    x = x[...,::-1]
    
    crop_scale = (0.85, 0.9)
    denoise_degree = 2
    rotate_angle = 1
    ksize = (7, 7)
    noise_param = 0.4
    
    if preprocess:
        if resize_func and for_all:
            x = resize_func(x, size, for_all)
        elif resize_func:
            x = resize_func(x, size)
            
        if 'joint' in aug_method:
            xd = x
            x = x.astype(np.float32)     
            x1 = random_crop(x, crop_scale)
            x2 = Denoiser.denoise_without_n(xd / 255, denoise_degree)
            x2 = x2.astype(np.float32)     
#             x2 = denoiser(x)
            x3 = rotate_crop(x, rotate_angle)
            x4 = cv2.blur(x, ksize)
            x5 = add_gauss_noise(xd, noise_param)
            x5 = x5.astype(np.float32)     
            x6 = sharpness(x)
            
            
            x1 = np.expand_dims(x1, axis=0)
            x2 = np.expand_dims(x2, axis=0)
            x3 = np.expand_dims(x3, axis=0)
            x4 = np.expand_dims(x4, axis=0)
            x5 = np.expand_dims(x5, axis=0)
            x6 = np.expand_dims(x6, axis=0)
            
            x1 = x1 / 255
            x2 = x2 / 255
            x3 = x3 / 255
            x4 = x4 / 255
            x5 = x5 / 255
            x6 = x6 / 255
            
            return x1, x2, x3, x4, x5, x6
        
        elif 'crop' in aug_method:
            x = x.astype(np.float32)  
            x = random_crop(x, crop_scale)
        elif 'denoise' in aug_method:
            x = Denoiser.denoise_without_n(x / 255, denoise_degree)
            x = x.astype(np.float32)  
#             x = denoiser(x)
        elif 'rotate' in aug_method:
            x = x.astype(np.float32)  
            x = rotate_crop(x, rotate_angle)
        elif 'blur' in aug_method:
            x = x.astype(np.float32)  
            x = cv2.blur(x, ksize)
        elif 'noise' in aug_method:
            x = add_gauss_noise(x, noise_param)
            x = x.astype(np.float32) 
        elif 'sharpness' in aug_method:
            x = x.astype(np.float32)  
            x = sharpness(x)
            
            
        x = np.expand_dims(x, axis=0)
        x = x / 255
        
    return x

def predict(x, y=None, model_gap=None, model=None, model_CNN=None, pca_cnn=None, pca_mg=None):
    '''
    Does prediction on given numpy image using
    model_gap and model
    '''
    try:
        feat_MG = model_gap.predict(x, verbose=0)
    except:
        x = x[None]
        feat_MG = model_gap.predict(x, verbose=0)
    if pca_mg:
        feat_MG = pca_mg.transform(feat_MG)
    if model_CNN:
        feat_CNN = model_CNN.predict(y, verbose=0)
        if pca_cnn:
            feat_CNN = pca_cnn.transform(feat_CNN)
        feat = np.concatenate((np.squeeze(feat_MG), np.squeeze(feat_CNN)))
        feat = feat[None]
    else:
        feat = feat_MG
    pred_score = model.predict(feat, verbose=0)

    return pred_score
    
def predict_from_path(model_gap, model, paths, resize_func=None, size=None, for_all=False, 
                      save_results=None, save_to=None, model_CNN=None, aug_method=None, pca_cnn=None, pca_mg=None):
    #always requires list of paths
    predicted = []
    
    for i, path in enumerate(paths):
        if aug_method =='joint':
            img_mg_1, img_mg_2, img_mg_3, img_mg_4, img_mg_5, img_mg_6 = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all, aug_method=aug_method)
        
            img_cnn_1 = None
            img_cnn_2 = None
            img_cnn_3 = None
            img_cnn_4 = None
            img_cnn_5 = None
            img_cnn_6 = None
            if model_CNN:
                img_cnn_1, img_cnn_2, img_cnn_3, img_cnn_4, img_cnn_5, img_cnn_6 = read_img(path=path, resize_func=resize_add_border, size=(600, 600), aug_method=aug_method)
                
            pred_score_1 = predict(img_mg_1, img_cnn_1, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            pred_score_2 = predict(img_mg_2, img_cnn_2, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            pred_score_3 = predict(img_mg_3, img_cnn_3, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            pred_score_4 = predict(img_mg_4, img_cnn_4, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            pred_score_5 = predict(img_mg_5, img_cnn_5, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            pred_score_6 = predict(img_mg_6, img_cnn_6, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            
            predicted.append(pred_score_1)
            predicted.append(pred_score_2)
            predicted.append(pred_score_3)
            predicted.append(pred_score_4)
            predicted.append(pred_score_5)
            predicted.append(pred_score_6)
            
        
        elif aug_method == 'crop' or aug_method == 'denoise' or aug_method == 'rotate' or  aug_method == 'blur'or aug_method == 'noise'or aug_method == 'sharpness':
            img_mg = read_img(path=path, resize_func=resize_func, size=size, for_all=for_all, aug_method=aug_method)
            
            img_cnn = None
            if model_CNN:
                img_cnn = read_img(path=path, resize_func=resize_add_border, size=(600, 600), aug_method=aug_method)    
            pred_score = predict(img_mg, img_cnn, model_gap, model, model_CNN, pca_cnn=pca_cnn, pca_mg=pca_mg)
            predicted.append(pred_score)
            
    predicted = np.array(predicted)
    predicted = np.squeeze(predicted)
    
    if save_results:
        np.save(save_to, np.argmax(predicted, axis=-1))
        
    return predicted

## Benchmark predictions

### Joint

In [7]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='joint', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [8]:
acc = calc_acc(labels_bench_joint, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 76.13 %


In [9]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_joint, pred)

Precision: 81.87 %
Recall: 56.03 %


### Rotate

In [10]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, size=(996, 996), 
                              model_CNN=model_CNN, aug_method='rotate', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [11]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 90.99 %


In [12]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 93.02 %
Recall: 85.11 %


### Crop

In [13]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='crop', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [14]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 88.29 %


In [15]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 85.42 %
Recall: 87.23 %


### Denoise

In [16]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='denoise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [17]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 83.78 %


In [18]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 73.02 %
Recall: 97.87 %


### Blur

In [19]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='blur', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [20]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 58.56 %


In [21]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 100.0 %
Recall: 2.13 %


### Noise

In [22]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='noise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [23]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 72.07 %


In [24]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 90.0 %
Recall: 38.3 %


### Sharpness

In [25]:
predicted = predict_from_path(model_gap, model, paths_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='sharpness', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [26]:
acc = calc_acc(labels_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 61.26 %


In [27]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench, pred)

Precision: 64.29 %
Recall: 19.15 %


## Benchmark2 predictions

### Joint

In [30]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='joint', pca_mg=pca_mg, pca_cnn=pca_cnn)


KeyboardInterrupt



In [12]:
acc = calc_acc(labels_bench_joint_2, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 0.0 %


C:\Users\CareAware\AppData\Local\Temp\ipykernel_3604\49106229.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  acc = np.sum(np.array(labels) == np.argmax(np.array(predicted), axis=1)) / len(labels)


In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_joint_2, pred)

### Rotate

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max,
                              size=(996, 996), model_CNN=model_CNN, aug_method='rotate', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

### Crop

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='crop', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

### Denoise

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='denoise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

### Blur

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='blur', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

### Noise

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='noise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

### Sharpness

In [ ]:
predicted = predict_from_path(model_gap, model, paths_bench_2, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='sharpness', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [ ]:
acc = calc_acc(labels_bench_2, predicted)
print(f'Accuracy: {acc} %')

In [ ]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_bench_2, pred)

## Benchmark predictions

### Joint

In [31]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='joint', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [32]:
acc = calc_acc(labels_alm_bench_joint, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 76.42 %


In [33]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench_joint, pred)

Precision: 90.13 %
Recall: 59.33 %


### Rotate

In [34]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max,
                              size=(996, 996), model_CNN=model_CNN, aug_method='rotate', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [35]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 96.0 %


In [36]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 95.1 %
Recall: 97.0 %


### Crop

In [37]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='crop', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [38]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 94.0 %


In [39]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 92.31 %
Recall: 96.0 %


### Denoise

In [40]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='denoise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [41]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 87.0 %


In [42]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 81.36 %
Recall: 96.0 %


### Blur

In [43]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='blur', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [44]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 51.0 %


In [45]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 100.0 %
Recall: 2.0 %


### Noise

In [46]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='noise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [47]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 70.0 %


In [48]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 97.62 %
Recall: 41.0 %


### Sharpness

In [49]:
predicted = predict_from_path(model_gap, model, paths_alm_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='sharpness', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [50]:
acc = calc_acc(labels_alm_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 60.0 %


In [51]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_alm_bench, pred)

Precision: 85.71 %
Recall: 24.0 %


## Custom data predictions

### Joint

In [9]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='joint', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [10]:
acc = calc_acc(labels_rand_bench_joint, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 75.42 %


In [11]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench_joint, pred)

Precision: 87.84 %
Recall: 59.0 %


### Rotate

In [13]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='rotate', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [14]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 93.5 %


In [15]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 95.79 %
Recall: 91.0 %


### Crop

In [16]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='crop', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [17]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 93.5 %


In [18]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 93.07 %
Recall: 94.0 %


### Denoise

In [19]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='denoise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [20]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 80.5 %


In [21]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 75.63 %
Recall: 90.0 %


### Blur

In [9]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='blur', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [10]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 49.0 %


In [11]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 0.0 %
Recall: 0.0 %


### Noise

In [11]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='noise', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [12]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 69.0 %


In [13]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 95.24 %
Recall: 40.0 %


### Sharpness

In [14]:
predicted = predict_from_path(model_gap, model, paths_rand_bench, resize_func=resize_max, 
                              size=(996, 996), model_CNN=model_CNN, aug_method='sharpness', pca_mg=pca_mg, pca_cnn=pca_cnn)

In [15]:
acc = calc_acc(labels_rand_bench, predicted)
print(f'Accuracy: {acc} %')

Accuracy: 67.0 %


In [16]:
#Precision and Recall
pred = np.argmax(predicted, axis=1)
calc_metrics(labels_rand_bench, pred)

Precision: 86.96 %
Recall: 40.0 %
